In [44]:
import nltk
import pandas as pd

In [52]:
!git clone https://github.com/cryptexcode/nlp_hackathon_bd_2023.git

Cloning into 'nlp_hackathon_bd_2023'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 16 (delta 2), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [53]:
!ls

LICENSE		 bengali_stemmer.egg-info  pytest.ini	 setup.py
README.rst	 build			   requirements  tests
bengali_stemmer  nlp_hackathon_bd_2023	   setup.cfg


# Bayesian to get started

In [55]:
ls nlp_hackathon_bd_2023/data

dev.txt  train.txt


In [52]:
def getDatasetFromIBOFile(filename): #nlp_hackathon_bd_2023/data/dev.txt"
    f = open(filename, "r")
    content = f.read()
    
    lines = content.split("\n")
    
    dataset = []
    for line in lines:
        parts = line.split(" _")
        word = parts[0]
        label = parts[len(parts)-1]

        dataset.append([word,label])
    return dataset


In [54]:
dataset = getDatasetFromIBOFile("nlp_hackathon_bd_2023/data/train.txt")
print(dataset[:10])
    

[['তার', ' O'], ['মৃত্যুর', ' O'], ['দশ', ' O'], ['দিন', ' O'], ['পর,', ' O'], ['১১৫', ' O'], ['কৃষ্ণাঙ্গ', ' O'], ['উচ্চ', ' O'], ['বিদ্যালয়ের', ' O'], ['শিক্ষার্থীরা', ' O']]


### EDA

In [60]:
unique_words = []
for row in dataset:
    if row[0] not in unique_words:
        unique_words.append(row[0])
print(len(unique_words), len(dataset))
print(unique_words[:10])

33186 207196
['তার', 'মৃত্যুর', 'দশ', 'দিন', 'পর,', '১১৫', 'কৃষ্ণাঙ্গ', 'উচ্চ', 'বিদ্যালয়ের', 'শিক্ষার্থীরা']


Disparity between total rows and unique words are very little. As far as bayesian is concerned, stemming poses no problem so lets stem!

In [9]:
# Installing Rafi Stemmer
!git clone https://github.com/banglakit/bengali-stemmer.git

fatal: destination path 'bengali-stemmer' already exists and is not an empty directory.


In [10]:
cd bengali-stemmer

/root/Getting_Started/bengali-stemmer


In [65]:
pip install .

Keyring is skipped due to an exception: 'keyring.backends'
Processing /root/Getting_Started/bengali-stemmer/bengali-stemmer
  Preparing metadata (setup.py) ... done
  Created wheel for bengali-stemmer: filename=bengali_stemmer-0.0.1-py2.py3-none-any.whl size=5535 sha256=a606c2188c5abf01271409cba103915560d02c3188466c48ed092a77d9676e2b
  Stored in directory: /root/.cache/pip/wheels/75/ad/fb/7fc7332eb3ce8779922870961801a525c9882b567709415b25
Successfully built bengali-stemmer
  Attempting uninstall: bengali-stemmer
    Found existing installation: bengali-stemmer 0.0.1
    Uninstalling bengali-stemmer-0.0.1:
      Successfully uninstalled bengali-stemmer-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [61]:
from bengali_stemmer.rafikamal2014 import RafiStemmer
stemmer = RafiStemmer()
stemmer.stem_word('বাংলায়')

unique_words = []
for row in dataset:
    stm = stemmer.stem_word(row[0])
    if stm not in unique_words:
        unique_words.append(stm)
print(len(unique_words), len(dataset))
print(unique_words[:10])

27823 207196
['ত', 'মৃত্যুর', 'দশ', 'দিন', 'পর,', '১১৫', 'কৃষ্ণাঙ্গ', 'উচ্চ', 'বিদ্যালয়', 'শিক্ষার্থী']


### Train

In [56]:
from collections import defaultdict

def train(dataset):
    yes_array = defaultdict(int)
    no_array = defaultdict(int)

    for row in dataset:
        word = row[0]
        label = row[1]

        if label == " O":
            no_array[word] += 1
        else:
            yes_array[word] += 1
    return yes_array, no_array

In [42]:
print(sorted(yes_array.items(), key=lambda x: x[1], reverse=True))        

[('', 799), ('অফ', 13), ('বিশ্ববিদ্যালয়', 11), ('দ্য', 11), ('তেল', 10), ('নিউ', 8), ('নদী', 7), ('কর্পোরেশন', 7), ('রেকর্ডস', 7), ('আমেরিকান', 7), ('ইউনিয়ন', 7), ('কোম্পানি', 7), ('পার্টি', 7), ('এফ.সি.', 6), ('ইয়র্ক', 6), ('ক্লাস', 6), ('ভ্যান', 5), ('ফুটবল', 5), ('জন্য', 5), ('ভ্যারাইটি', 4), ('দল', 4), ('ক্লাব', 4), ('কাউন্টি', 4), ('সেন্ট', 4), ('রটেন', 4), ('টম্যাটোস', 4), ('মিউজিক', 4), ('চলচ্চিত্র', 4), ('জাতীয়', 4), ('জ্বালানি', 4), ('এবং', 4), ('সিটি', 4), ('অ্যান্ড', 4), ('টমাস', 4), ('ব্রিটিশ', 4), ('রেল', 4), ('জর্জ', 4), ('ইউনিভার্সিটি', 4), ('৪', 3), ('মার্কিন', 3), ('ইউনাইটেড', 3), ('লুই', 3), ('বেসবল', 3), ('কেন্দ্রীয়', 3), ('লন্ডন', 3), ('ফিল্ডটর্ফ', 3), ('ডিভিডি', 3), ('এমটিভি', 3), ('মার্টিন', 3), ('প্রেস', 3), ('সোসাইটি', 3), ('রাগবি', 3), ('লীগ', 3), ('ইন্টারন্যাশনাল', 3), ('মোটর', 3), ('রেলওয়ে', 3), ('২', 3), ('অ্যালেন', 3), ('লাইব্রেরি', 3), ('গ্রুপ', 3), ('চেয়ার', 3), ('কানাডিয়ান', 3), ('ব্রডকাস্টিং', 3), ('আন্তর্জাতিক', 3), ('মাংস', 3), ('স্কুল', 3), (

In [43]:
print(sorted(no_array.items(), key=lambda x: x[1], reverse=True))

[('এবং', 237), ('এর', 192), ('একটি', 183), ('তিনি', 125), ('তার', 108), ('সালে', 105), ('করা', 103), ('জন্য', 102), ('এটি', 91), ('এ', 73), ('হয়েছিল।', 66), ('থেকে', 65), (',', 61), ('দ্বারা', 60), ('।', 58), ('হয়।', 54), ('সাথে', 51), ('যা', 48), ('-', 47), ('এই', 46), ('করে', 44), ('করেন।', 37), ('হিসেবে', 36), ('করে।', 34), ('যে', 32), ('তাদের', 31), ('ব্যবহার', 31), ('ছিল', 30), ('মধ্যে', 28), ('করার', 28), ('প্রথম', 26), ('হয়', 26), ('দিয়ে', 24), ('কাজ', 24), ('একজন', 24), ('বা', 24), ('ছিল।', 23), ('সালে,', 23), ('তৈরি', 22), ('যার', 22), ('হন।', 21), ('পর্যন্ত', 20), ('হয়েছে।', 19), ('সালের', 19), ('করতে', 19), ('নতুন', 18), ('পর', 18), ('ছিলেন।', 17), ('হয়েছিল', 17), ('ব্যবহৃত', 17), ('শুরু', 17), ('নিয়ে', 16), ('কিছু', 16), ('প্রধান', 16), ('করেছিলেন।', 16), ('তারা', 16), ('তাকে', 16), ('কিন্তু', 16), ('করেছেন।', 15), ('বর্ণনা', 15), ('পরে', 15), ('আগে', 15), ('টি', 15), ('সময়', 14), ('বছর', 14), ('হল', 14), ('রয়েছে।', 14), ('করে,', 14), ('দিকে', 13), ('হিসাবে', 13), 

In [57]:
def predict(word, yes_array, no_array):
    if no_array[word]==0:
        return 1
    T = yes_array[word]/no_array[word]
    
    if T>0.4:
        return 1
    else:
        return 0

In [58]:
predict("প্রার্থনার", yes_array, no_array)

0

In [59]:
predict("কোয়ার্টজ", yes_array, no_array)

1

### Evaluating Bayesian

In [64]:
train_dataset = getDatasetFromIBOFile("nlp_hackathon_bd_2023/data/train.txt")
test_dataset = getDatasetFromIBOFile("nlp_hackathon_bd_2023/data/dev.txt")

In [67]:
yes_array, no_array = train(train_dataset)

In [80]:
tp = 0
tn = 0
fp = 0
fn = 0

for row in test_dataset: 
    word = row[0]
    actual_label = 0
    if row[1] == " O": actual_label = 0
    else: actual_label = 1
    
    predicted_label = predict(word, yes_array, no_array)
    
    match = actual_label==predicted_label
    #print("DEBUG:"+str(match))
    
    if match==0 and predicted_label==0: fn+=1
    elif match==0 and predicted_label==1: fp+=1
    elif match==1 and predicted_label==0: tn+=1
    elif match==1 and predicted_label==1: tp += 1

accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp+fp)
recall = tp/(tp+fn)
f_score = 2*(precision*recall)/*
print("Train dataset size is around "+str(len(yes_array)+len(no_array)))
print("Test dataset size is around "+str(len(test_dataset)))

print("Accuracy:"+str(accuracy))
print("precision:"+str(precision))
print("recall:"+str(recall))
print("F-score:"+)

Train dataset size is around 40064
Test dataset size is around 11132
Accuracy:0.873697448796263
precision:0.6570926143024619
recall:0.9047619047619048
